In [3]:
import numpy as np
!pip install neupy
from neupy import algorithms

def draw_bin_image(image_matrix):
    for row in image_matrix.tolist():
        print('| ' + ' '.join(' *'[val] for val in row))

zero = np.matrix([
    0, 1, 1, 1, 0,
    1, 0, 0, 0, 1,
    1, 0, 0, 0, 1,
    1, 0, 0, 0, 1,
    1, 0, 0, 0, 1,
    0, 1, 1, 1, 0
])

one = np.matrix([
    0, 1, 1, 0, 0,
    0, 0, 1, 0, 0,
    0, 0, 1, 0, 0,
    0, 0, 1, 0, 0,
    0, 0, 1, 0, 0,
    0, 0, 1, 0, 0
])

two = np.matrix([
    1, 1, 1, 0, 0,
    0, 0, 0, 1, 0,
    0, 0, 0, 1, 0,
    0, 1, 1, 0, 0,
    1, 0, 0, 0, 0,
    1, 1, 1, 1, 1,
])

draw_bin_image(zero.reshape((6, 5)))

    100% |████████████████████████████████| 204kB 12.2MB/s a 0:00:01
    100% |████████████████████████████████| 829kB 17.2MB/s ta 0:00:01
  Running setup.py bdist_wheel for tableprint ... done
  Stored in directory: /Users/precious.enahoro/Library/Caches/pip/wheels/f3/85/50/13fdeeb11b9453c850be1b5b964471e05d5b9454d8c3c56e76
  Running setup.py bdist_wheel for future ... done
  Stored in directory: /Users/precious.enahoro/Library/Caches/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built tableprint future
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


|   * * *  
| *       *
| *       *
| *       *
| *       *
|   * * *  


In [4]:
data = np.concatenate([zero, one, two], axis=0)

dhnet = algorithms.DiscreteHopfieldNetwork(mode='sync')
dhnet.train(data)

In [5]:
half_zero = np.matrix([
    0, 1, 1, 1, 0,
    1, 0, 0, 0, 1,
    1, 0, 0, 0, 1,
    0, 0, 0, 0, 0,
    0, 0, 0, 0, 0,
    0, 0, 0, 0, 0,
])
draw_bin_image(half_zero.reshape((6, 5)))







half_two = np.matrix([
    0, 0, 0, 0, 0,
    0, 0, 0, 0, 0,
    0, 0, 0, 0, 0,
    0, 1, 1, 0, 0,
    1, 0, 0, 0, 0,
    1, 1, 1, 1, 1,
])
draw_bin_image(half_two.reshape((6, 5)))


|   * * *  
| *       *
| *       *
|          
|          
|          
|          
|          
|          
|   * *    
| *        
| * * * * *


In [6]:
result = dhnet.predict(half_zero)
draw_bin_image(result.reshape((6, 5)))







result = dhnet.predict(half_two)
draw_bin_image(result.reshape((6, 5)))



|   * * *  
| *       *
| *       *
| *       *
| *       *
|   * * *  
| * * *    
|       *  
|       *  
|   * *    
| *        
| * * * * *


In [7]:
half_two = np.matrix([
    1, 1, 1, 0, 0,
    0, 0, 0, 1, 0,
    0, 0, 0, 1, 0,
    0, 0, 0, 0, 0,
    0, 0, 0, 0, 0,
    0, 0, 0, 0, 0,
])

result = dhnet.predict(half_two)
draw_bin_image(result.reshape((6, 5)))



|   * *    
|     *    
|     *    
|   * *    
| *   *    
| * * * * *


In [8]:
from neupy import environment
environment.reproducible()

dhnet.mode = 'async'
dhnet.n_times = 400

result = dhnet.predict(half_two)
draw_bin_image(result.reshape((6, 5)))






result = dhnet.predict(half_two)
draw_bin_image(result.reshape((6, 5)))

|   * *    
|     *    
|     *    
|     *    
|     *    
|     *    
| * * *    
|       *  
|       *  
|   * *    
| *        
| * * * *  


In [9]:
A = """
.XXX.
X...X
XXXXX
X...X
X...X
"""

Z = """
XXXXX
...X.
..X..
.X...
XXXXX
"""

In [10]:
def to_pattern(letter):
    from numpy import array
    return array([+1 if c=='X' else -1 for c in letter.replace('\n','')])

In [11]:
def display(pattern):
    from pylab import imshow, cm, show
    imshow(pattern.reshape((5,5)),cmap=cm.binary, interpolation='nearest')
    show()

In [13]:
patterns = np.array([to_pattern(A), to_pattern(Z)])

  
def train(patterns):
    from numpy import zeros, outer, diag_indices 
    r,c = patterns.shape
    W = zeros((c,c))
    for p in patterns:
        W = W + outer(p,p)
    W[diag_indices(c)] = 0
    return W/r

In [14]:
def recall(W, patterns, steps=5):
    from numpy import vectorize, dot
    sgn = vectorize(lambda x: -1 if x<0 else +1)
    for _ in xrange(steps):        
        patterns = sgn(dot(patterns,W))
    return patterns

In [15]:
def hopfield_energy(W, patterns):
    from numpy import array, dot
    return array([-0.5*dot(dot(p.T,W),p) for p in patterns])


In [18]:
def create_memory(size):
    return [np.random.randint(2,size=size) for i in range(size)]

def capacity(size):
    counter = 0
    memory = create_memory(size)
    W= np.zeros((size,size))
    train(patterns)
    recall(W,memory)
    if recall ==1;
        counter +=1
    return counter/size
    

def increase_capacity():
    size = 1
    cap= capacity(size)
    while np.all(cap == 1) and size < 80:
        size += 1
        cap = capacity(size)
    return size

#increase_capacity()

In [27]:
size =5
[np.random.randint(2,size=size) for i in range(size)]

[array([0, 0, 0, 0, 0]),
 array([0, 0, 0, 1, 0]),
 array([1, 0, 1, 0, 0]),
 array([0, 1, 0, 1, 1]),
 array([0, 0, 0, 1, 0])]

Write a function to determine the capacity for a Hopfield network of a given size. This function should have a size parameter, and then initialize an empty Hopfield network. It must then store a random memory and test whether the network is able to recall the correct memory if given a pattern corrupted by a single bit. The function must then return the largest number of memories that were stored and successfully recalled.

In [ ]:
def capacity(size):
    empty = np.zeros(size, size)
    